In [1]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Load the data in one-hot format:

In [31]:
gmc_forms = []
for l in open('gmc_aligned.tsv','r'):
    gmc_forms.append(l.split('\t'))

In [45]:
def gen_data():
    """load aligned forms"""
    gmc_forms = []
    for l in open('gmc_aligned.tsv','r'):
        gmc_forms.append(l.split('\t'))
    """"""
    gmc_forms=[l for l in gmc_forms if l[0]=='ang']
    x_raw = [l[1].split() for l in gmc_forms]
    y_raw = [l[2].split() for l in gmc_forms]
    """length of set of symbols for input and output"""
    x_alpha = sorted(set([s for w in x_raw for s in w]))
    y_alpha = sorted(set([s for w in y_raw for s in w]))
    """max lengh (for padding)"""
    x_max = max([len(w) for w in x_raw])
    y_max = max([len(w) for w in y_raw])
    """"""
    N = len(x_raw)
    """"""
    x = np.zeros([N,x_max,len(x_alpha)],dtype='float32')
    y = np.zeros([N,y_max,len(y_alpha)],dtype='float32')
    """"""
    for i,w in enumerate(x_raw):
        for j,s in enumerate(w):
            x[i,j,x_alpha.index(s)] = 1
    """"""
    for i,w in enumerate(y_raw):
        for j,s in enumerate(w):
            y[i,j,y_alpha.index(s)] = 1
    cutoff = int(.9*len(x))
    x_train = x[:cutoff,:,:]
    x_test = x[cutoff:,:,:]
    y_train = y[:cutoff,:,:]
    y_test = y[cutoff:,:,:]
    return(x_train,y_train,x_test,y_test,x_max,y_max,N)




x_train,y_train,x_test,y_test,x_max,y_max,N = gen_data()

In [47]:
X = x_train.shape[2]
Y = y_train.shape[2]
L = y_train.shape[1]
N = x_train.shape[0]
Z = 10

print('Number of input symbols: {}'.format(X))
print('Number of output symbols: {}'.format(Y))
print('Maximum sequence length: {}'.format(L))
print('Number of string pairs: {}+{}'.format(x_train.shape[0],x_test.shape[0]))

Number of input symbols: 34
Number of output symbols: 40
Maximum sequence length: 29
Number of string pairs: 5588+621


Define a feed-forward neural network:

In [48]:
x_ = tf.placeholder(tf.float32,shape=(None,L,X))
y_ = tf.placeholder(tf.float32,shape=(None,L,Y))
U = tf.Variable(tf.random_normal([X,Z]))
hidden = tf.nn.softmax(tf.tensordot(x_,U,axes=1))            #L by Z matrix
W = tf.Variable(tf.random_normal([Z,Y]))
output_layer = tf.tensordot(hidden,W,axes=1)                 #L by Y matrix

In [49]:
lr = tf.placeholder(tf.float32, shape=(), name='learning_rate')
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output_layer, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

In [50]:
pred = tf.nn.softmax(output_layer)
pred_label = tf.argmax(pred,1)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [51]:
# Create operation which will initialize all variables
init = tf.global_variables_initializer()

# Configure GPU not to use all memory
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Start a new tensorflow session and initialize variables
sess = tf.InteractiveSession(config=config)
sess.run(init)

In [53]:
batch_size=32
training_epochs=500
display_step=1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, loss], 
                            feed_dict={
                                x_: batch_x, 
                                y_: batch_y
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "loss=", \
                "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

Epoch: 0001 loss= 1.122533042
Epoch: 0002 loss= 1.116518313
Epoch: 0003 loss= 1.110571521
Epoch: 0004 loss= 1.104687836
Epoch: 0005 loss= 1.098867050
Epoch: 0006 loss= 1.093107848
Epoch: 0007 loss= 1.087408536
Epoch: 0008 loss= 1.081767051
Epoch: 0009 loss= 1.076181161
Epoch: 0010 loss= 1.070648859
Epoch: 0011 loss= 1.065168915
Epoch: 0012 loss= 1.059741079
Epoch: 0013 loss= 1.054365574
Epoch: 0014 loss= 1.049042596
Epoch: 0015 loss= 1.043771623
Epoch: 0016 loss= 1.038551374
Epoch: 0017 loss= 1.033379488
Epoch: 0018 loss= 1.028253002
Epoch: 0019 loss= 1.023169256
Epoch: 0020 loss= 1.018126081
Epoch: 0021 loss= 1.013122070
Epoch: 0022 loss= 1.008156263
Epoch: 0023 loss= 1.003227960
Epoch: 0024 loss= 0.998336611
Epoch: 0025 loss= 0.993481652
Epoch: 0026 loss= 0.988662663
Epoch: 0027 loss= 0.983879193
Epoch: 0028 loss= 0.979130704
Epoch: 0029 loss= 0.974416885
Epoch: 0030 loss= 0.969737257
Epoch: 0031 loss= 0.965091488
Epoch: 0032 loss= 0.960479182
Epoch: 0033 loss= 0.955900020
Epoch: 003

Epoch: 0275 loss= 0.474010404
Epoch: 0276 loss= 0.473524879
Epoch: 0277 loss= 0.473043800
Epoch: 0278 loss= 0.472567094
Epoch: 0279 loss= 0.472094722
Epoch: 0280 loss= 0.471626607
Epoch: 0281 loss= 0.471162681
Epoch: 0282 loss= 0.470702903
Epoch: 0283 loss= 0.470247190
Epoch: 0284 loss= 0.469795478
Epoch: 0285 loss= 0.469347747
Epoch: 0286 loss= 0.468903870
Epoch: 0287 loss= 0.468463847
Epoch: 0288 loss= 0.468027600
Epoch: 0289 loss= 0.467595070
Epoch: 0290 loss= 0.467166206
Epoch: 0291 loss= 0.466740970
Epoch: 0292 loss= 0.466319287
Epoch: 0293 loss= 0.465901130
Epoch: 0294 loss= 0.465486426
Epoch: 0295 loss= 0.465075145
Epoch: 0296 loss= 0.464667215
Epoch: 0297 loss= 0.464262598
Epoch: 0298 loss= 0.463861231
Epoch: 0299 loss= 0.463463091
Epoch: 0300 loss= 0.463068135
Epoch: 0301 loss= 0.462676312
Epoch: 0302 loss= 0.462287561
Epoch: 0303 loss= 0.461901855
Epoch: 0304 loss= 0.461519154
Epoch: 0305 loss= 0.461139422
Epoch: 0306 loss= 0.460762590
Epoch: 0307 loss= 0.460388649
Epoch: 030

In [54]:
print("Accuracy:", accuracy.eval({x_: x_test, y_: y_test}))

Accuracy: 0.15877616


In [ ]:
help(tf.reduce_mean)

In [56]:
U.eval()

array([[-0.4045834 , -1.3274912 ,  0.8956847 , -1.7651782 ,  1.0364856 ,
        -1.2952708 , -1.5942513 , -0.47434276, -1.8660967 ,  0.23347509],
       [-1.2535244 , -1.5511212 ,  1.009658  ,  0.21823516, -0.0832367 ,
         2.153867  , -0.4979945 , -0.95245534,  0.43519378, -0.3843823 ],
       [ 0.27929285, -0.40438697, -0.87659645, -0.87517154,  0.26784536,
         1.4496769 ,  0.17188928, -0.21270004, -1.2267426 ,  0.26768878],
       [-0.27646998, -0.45929882,  0.02620709,  0.6652723 , -0.5112964 ,
        -0.5047536 ,  1.3499221 , -0.84636456, -0.3131396 , -0.19102888],
       [ 0.64400893, -0.92748016, -1.8711928 ,  0.20264588, -1.4982809 ,
        -0.49323344, -0.00744544,  0.28357384,  0.5164095 ,  1.3628272 ],
       [-1.687928  ,  1.3751528 ,  0.07957733,  0.25146568,  2.2168996 ,
         0.6300091 ,  1.2901114 ,  1.7603753 , -0.34151906, -0.66972876],
       [ 0.7972903 , -0.6933182 ,  1.2402503 ,  0.06241497,  0.8922171 ,
        -1.3946333 ,  0.44395125, -1.1306434 